In [66]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage.interpolation import zoom
import zlib
import blosc

In [27]:
def Depth2Gray(d4d):
        d4d = 255 - cv2.cvtColor(d4d, cv2.COLOR_GRAY2RGB)
        return d4d

## Easy Intra-frame Compression Methods

### Test Compression Tools for Depth and Color Images

In [59]:
def depth_compress(i):
    name = "apple_1/apple_1_1_"
    filename = name+str(i)+"_depth.png"
    depth = cv2.imread(filename)
    
    return len(depth.tostring()), len(zlib.compress(depth.tostring())), len(blosc.compress(depth.tostring()))
def color_compress(i):
    name = "apple_1/apple_1_1_"
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    return len(color.tostring()), len(zlib.compress(color.tostring())), len(blosc.compress(color.tostring()))

In [63]:
%timeit
td=zd=bd=tc=zc=bc = 0
for i in range(1, 209):
    t,z,b = depth_compress(i)
    td += t
    zd += z
    bd += b
    t,z,b = color_compress(i)
    tc += t
    zc += z
    bc += b
td /= 208.0
bd /= 208.0
zd /= 208.0
tc /= 208.0
zc /= 208.0
bc /= 208.0
print("Depth:")
print("Average Date size：",td)
print("After Native Zlib Compression:",zd , 
      "; Compression Rate:",zd/td*100)
print("After Blosc Compression:", bd, 
      "; Compression Rate:",bd/td*100)
print("Color:")
print("Average Data size: ", tc)
print("After Native Zlib Compression:",zc , 
      "; Compression Rate:",zc/tc*100)
print("After Blosc Compression:", bc, 
      "; Compression Rate:",bc/tc*100)

Depth:
Average Date size： 921600.0
After Native Zlib Compression: 13736.033653846154 ; Compression Rate: 1.490455040564904
After Blosc Compression: 127634.97596153847 ; Compression Rate: 13.849281245826656
Color:
Average Data size:  921600.0
After Native Zlib Compression: 444878.9711538461 ; Compression Rate: 48.27245780749199
After Blosc Compression: 848393.1346153846 ; Compression Rate: 92.05654672475961


### Test the compression rate if transmit them together

In [64]:
def rgbd_compress(i):
    name = "apple_1/apple_1_1_"
    filename = name+str(i)+"_depth.png"
    depth = cv2.imread(filename)
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    rgbd = np.dstack((color, depth))
    return len(rgbd.tostring()), len(zlib.compress(rgbd.tostring())), len(blosc.compress(rgbd.tostring()))
td=zd=bd = 0
for i in range(1, 209):
    t,z,b = rgbd_compress(i)
    td += t
    zd += z
    bd += b
td /= 208.0
bd /= 208.0
zd /= 208.0
print("RGBD:")
print("Average Date size：",td)
print("After Native Zlib Compression:",zd , 
      "; Compression Rate:",zd/td*100)
print("After Blosc Compression:", bd, 
      "; Compression Rate:",bd/td*100)

RGBD:
Average Date size： 1843200.0
After Native Zlib Compression: 542334.9182692308 ; Compression Rate: 29.42355242346087
After Blosc Compression: 1293149.5625 ; Compression Rate: 70.15785386827257


### Lossy Intra-frame Compression

In [79]:
def depth_compress_lossy(i):
    name = "apple_1/apple_1_1_"
    filename = name+str(i)+"_depth.png"
    depth = cv2.imread(filename)
    depth = zoom(depth, [0.5,0.5,1])
    n_depth = np.std((zoom(depth, [2,2,1]) - cv2.imread(filename)))
    return len(depth.tostring()), len(zlib.compress(depth.tostring())), len(blosc.compress(depth.tostring())), n_depth
def color_compress_lossy(i):
    name = "apple_1/apple_1_1_"
    filename = name+str(i)+".png"
    color = cv2.imread(filename)
    color = zoom(color,[0.5,0.5,1])
    n_color = np.std((zoom(color,[2,2,1]) - cv2.imread(filename)))
    return len(color.tostring()), len(zlib.compress(color.tostring())), len(blosc.compress(color.tostring())), n_color

In [80]:
td_l=zd_l=bd_l=tc_l=zc_l=bc_l=nd=nc = 0
for i in range(1, 209):
    t,z,b,n = depth_compress_lossy(i)
    td_l += t
    zd_l += z
    bd_l += b
    nd += n
    t,z,b,n = color_compress_lossy(i)
    tc_l += t
    zc_l += z
    bc_l += b
    nc += n
td_l /= 208.0
bd_l /= 208.0
zd_l /= 208.0
tc_l /= 208.0
zc_l /= 208.0
bc_l /= 208.0
nd /= 208.0
nc /= 208.0
print("Depth:")
print("Average Date size：",td_l)
print("After Native Zlib Compression:",zd_l , 
      "; Compression Rate:",zd_l/td_l*100)
print("After Blosc Compression:", bd_l, 
      "; Compression Rate:",bd_l/td_l*100)
print("Lossy rate: Std of difference: ", nd)
print("Color:")
print("Average Data size: ", tc_l)
print("After Native Zlib Compression:",zc_l , 
      "; Compression Rate:",zc_l/tc_l*100)
print("After Blosc Compression:", bc_l, 
      "; Compression Rate:",bc_l/tc_l*100)
print("Lossy rate: Std of difference: ", nc)

Depth:
Average Date size： 230400.0
After Native Zlib Compression: 6415.379807692308 ; Compression Rate: 2.7844530415331197
After Blosc Compression: 41133.75961538462 ; Compression Rate: 17.853194277510685
Lossy rate: Std of difference:  34.0414872736
Color:
Average Data size:  230400.0
After Native Zlib Compression: 160165.72596153847 ; Compression Rate: 69.51637411525107
After Blosc Compression: 219063.8798076923 ; Compression Rate: 95.0798089443109
Lossy rate: Std of difference:  112.724191067
